<a href="https://colab.research.google.com/github/mohitsinghdz/kaggle_solutions/blob/master/LANL_Earthquake_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install kaggle

In [4]:
!pip install numpy==1.15.0


    100% |████████████████████████████████| 13.9MB 3.0MB/s 
torchvision 0.2.2.post2 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.2 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


In [5]:
!pip install catboost

    100% |████████████████████████████████| 55.5MB 732kB/s 


In [0]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt

In [33]:
!unzip test.zip


Archive:  test.zip
replace seg_430e66.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:

scaler = StandardScaler()


importing dependecies


In [0]:
from google.colab import files


uploaded = files.upload()

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c LANL-Earthquake-Prediction


In [2]:
!unzip train.csv.zip


Archive:  train.csv.zip
  inflating: train.csv               


In [43]:
test =  pd.read_csv('seg_0012b5.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

X_train_test = pd.DataFrame()
y_train_test = pd.Series()
for df in test:
  #  print(df)
    ch = gen_features(df['acoustic_data'])
    X_train_test = X_train_test.append(ch, ignore_index=True)
  #  y_train_test = y_train_test.append(pd.Series(df['time_to_failure'].values[-1]))
print(X_train_test)
  #X_train_test.describe()

         0         1      2          3         4     5    6     7     8   \
0  4.171153  5.922839 -140.0  56.218955  0.295708 -12.0 -2.0  11.0  20.0   

      9        10        11  
0  152.0  5.19834  5.045369  


In [0]:

train = pd.read_csv('train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
#function to generate statistical features
def gen_features(X): 
  strain = []
  strain.append(X.mean())
  strain.append(X.std())
  strain.append(X.min())
  strain.append(X.kurtosis())
  strain.append(X.skew())
  strain.append(np.quantile(X,0.01))
  strain.append(np.quantile(X,0.05))
  strain.append(np.quantile(X,0.95))
  strain.append(np.quantile(X,0.99))
  strain.append(np.abs(X).max())
  strain.append(np.abs(X).mean())
  strain.append(np.abs(X).std())
  return pd.Series(strain)
X_train = pd.DataFrame()
y_train = pd.Series()
for df in train:
   
    ch = gen_features(df['acoustic_data'])
    X_train = X_train.append(ch, ignore_index=True)
    y_train = y_train.append(pd.Series(df['time_to_failure'].values[-1]))
X_train.describe()

In [30]:
train_pool = Pool(X_train,y_train) #to collect training data, input and output
m = CatBoostRegressor(iterations=10000, loss_function='MAE',boosting_type="Ordered")  #mean absolute error
m.fit(X_train,y_train, silent=True)
m.best_score_

{'learn': {'MAE': 1.7821816142256932}}

In [100]:
#testing the model
sub = pd.read_csv('sample_submission.csv', nrows=2624, iterator=True, chunksize=1,  dtype={ 'seg_id': np.str, 'time_to_failure': np.float64})

X_train_test = pd.DataFrame()
#y_train = pd.Series()
for df,index in enumerate(sub):
    #read file for test
    test =  pd.read_csv(index['seg_id'][df]+'.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

    
    for df in test:
  #  print(df)
        ch = gen_features(df['acoustic_data'])
        X_train_test = X_train_test.append(ch, ignore_index=True)
  #  y_train_test = y_train_test.append(pd.Series(df['time_to_failure'].values[-1]))
       # print(X_train_test)
        
      #trying something new
        #sol = m.predict(X_train_test,ntree_start=0,ntree_end=0,thread_count=-1,verbose=None)
        #sub['time_to_failure'][df] = float(sol)
#print(sub)
#tried something new
#X_train_test = scaler.transform(X_train_test)
#X_train_test = X_train_test.reshape(2624,13,1)
pred = m.predict(X_train_test)
print(pred)
#for df,index in enumerate(sub):
#  index['time_to_failure'][df] = pred[df]
#sub['time_to_failure'] = pred
#sub.to_csv('submission.csv')
np.savetxt("foo.csv", pred, delimiter=",")

[ 4.62752045  4.91808233  4.40467357 ...  4.92529933  1.75204515
 10.50246411]


In [0]:
test =  pd.read_csv('seg_0012b5.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
print (test)
X_train_test = pd.DataFrame()
y_train_test = pd.Series()
for df in test:
  #  print(df)
    ch = gen_features(df['acoustic_data'])
    X_train_test = X_train_test.append(ch, ignore_index=True)
  #  y_train_test = y_train_test.append(pd.Series(df['time_to_failure'].values[-1]))
print(X_train_test)
  #X_train_test.describe()
  
  
m.predict(X_train_test,ntree_start=0,ntree_end=0,thread_count=-1,verbose=None)